## Loading, Chunking

In [7]:
from langchain.document_loaders import DirectoryLoader, TextLoader, CSVLoader

# QA Data
loader = CSVLoader("qa_data.csv")
qa_documents = loader.load()

In [8]:
# Annual Report Data
path = '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023_2.txt'
loader = TextLoader(path)
ar_documents = loader.load()

## Chunking
import re #정규표현식
from langchain.text_splitter import RecursiveCharacterTextSplitter #스플리터

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,   # 원하는 청크 크기 설정
    chunk_overlap=50,  # 중첩되는 토큰 수
    separators=["\n\n", "\n", ". ", "? ", "! "]  # 사용자 정의 분할 함수 추가
)

splitted_ar_documents = text_splitter.split_documents(ar_documents)

# 분할 후 각 문서의 내용을 정리. 문장이 마침표로 시작하면 마침표 제거, 문장이 마침표로 끝나지 않으면 마침표 추가.
for i in range(len(splitted_ar_documents)):
    splitted_ar_documents[i].page_content = splitted_ar_documents[i].page_content.strip('. ').strip()
    if not splitted_ar_documents[i].page_content.endswith('.'):
        splitted_ar_documents[i].page_content += '.'

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
ar_texts = [doc.page_content for doc in splitted_ar_documents]        

## Embedding

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
from langchain import FAISS
import numpy as np

encoder_model = SentenceTransformer('jeonseonjin/embedding_BAAI-bge-m3')

# 질문-답변 FAISS 인덱스를 임베딩
questions = [doc.page_content for doc in qa_documents] #질문쌍만 따로 추출..을 해야 하는데 못했음
question_embeddings = encoder_model.encode(questions) #전체 QA 쌍을 임베딩으로 변환
index_dim = question_embeddings.shape[1] #변환한 객체의 1024 차원
question_index = faiss.IndexFlatIP(index_dim)  #1024 차원 faiss index 객체
question_index.add(question_embeddings) #변환한 객체를 index에 추가
faiss.write_index(question_index, 'question_index.bin') #QA index

# 답변 텍스트 FAISS 인덱스를 별도로 임베딩
answers = [doc.page_content[1] for doc in qa_documents] #질문쌍만 따로 추출
answer_embeddings = encoder_model.encode(answers) #전체 질문쌍을 임베딩으로 변환
answer_index = faiss.IndexFlatIP(index_dim) #1024 차원 faiss index 객체
answer_index.add(answer_embeddings) #변환한 객체를 index에 추가
faiss.write_index(answer_index, 'answer_index.bin') #A index

# 연차보고서 FAISS 인덱스를 임베딩
ar_embeddings = encoder_model.encode(ar_texts)
ar_index = faiss.IndexFlatIP(index_dim)  
ar_index.add(ar_embeddings)
faiss.write_index(ar_index, 'ar_index.bin')

In [10]:
# FAISS 인덱스에 모든 임베딩 추가 (Inner Product 방식 사용)
combined_index = faiss.IndexFlatIP(index_dim)  # IP: 내적 기반 검색
combined_index.add(np.array(question_embeddings))
combined_index.add(np.array(answer_embeddings))
combined_index.add(np.array(ar_embeddings))

# 출처 구분을 위한 인덱스 범위 설정
num_questions = len(questions)
num_answers = len(answers)
num_annual_reports = len(ar_texts)

## Retrieval

In [11]:
def search_and_respond(query, encoder_model=encoder_model, question_index=question_index, answer_index=answer_index, ar_index=ar_index, answers=answers, ar_texts=ar_texts, critical_value_1=20, critical_value_2=40):
    # 질문에 대한 임베딩 생성
    question_embedding = encoder_model.encode([query])

    # 질문에 대해 question_index에서 검색
    distance_questions, index_questions = question_index.search(np.array(question_embedding), k=1)
    similarity_questions = 100/(1+distance_questions[0][0])  # 거리 값 그대로 사용

    print(f"질문에 대한 거리: {distance_questions:.4f} (임계값 1: {critical_value_1}, 임계값 2: {critical_value_2})")
    
    if similarity_questions > critical_value_1:
        # 유사한 질문이 있을 경우 해당 질문에 매칭된 답변 출력
        closest_question = questions[index_questions[0][0]]
        corresponding_answer = answers[index_questions[0][0]]
        
        print(f"질문에서 찾은 최근접 문장의 거리: {distance_questions:.4f}")
        print(f"가장 가까운 질문: {closest_question}")
        print(f"해당 질문에 대한 답변: {corresponding_answer}")
    
    # 유사도가 critical_value_1 아래일 경우, 답변에서 재검색
    elif similarity_answers > critical_value_2:
        print("질문에서 유사한 질문을 찾지 못했습니다. 답변에서 다시 검색합니다.")
        
        # 답변에서 검색
        distance_answers, index_answers = answer_index.search(np.array(question_embedding), k=1)
        similarity_questions  = distance_answers[0][0]  # 답변 최근접 문장의 거리 계산
        
        print(f"답변에서 찾은 최근접 문장의 거리: {answer_distance:.4f}")
        print(f"답변: {answers[index_answers[0][0]]}")
    
    # 유사도가 critical_value_2 아래일 경우, 연차보고서에서 재검색
    else:
        print("질문에서 유사한 답변을 찾지 못했습니다. 연차보고서에서 다시 검색합니다.")
        
        # 연차보고서에서 검색
        distance_ar, index_ar = ar_index.search(np.array(question_embedding), k=1)
        ar_distance = distance_ar[0][0]  # 보고서 최근접 문장의 거리 계산
        
        print(f"연차보고서에서 찾은 최근접 문장의 거리: {ar_distance:.4f}")
        print(f"연차보고서 내용: {ar_texts[index_ar[0][0]]}")

# 함수 호출 예시
search_and_respond('물가안정목표제에 대해')


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [ ]:
def search_combined_index(query, encoder_model=encoder_model, combined_index=combined_index, num_questions=num_questions, num_answers=num_answers, questions=questions, answers=answers, ar_texts=ar_texts):
    # 질문에 대한 임베딩 생성
    question_embedding = encoder_model.encode([query])

    # 통합 인덱스에서 검색
    distance_combined, index_combined = combined_index.search(np.array(question_embedding), k=1)
    combined_distance = distance_combined[0][0]  # 가장 가까운 문장의 거리 계산

    # 가장 가까운 문장의 출처 및 출력
    closest_index = index_combined[0][0]

    if closest_index < num_questions:
        source = '질문'
        result_text = questions[closest_index]
    elif closest_index < num_questions + num_answers:
        source = '답변'
        result_text = answers[closest_index - num_questions]
    else:
        source = '연차보고서'
        result_text = ar_texts[closest_index - num_questions - num_answers]

    # 결과 출력
    print(f"가장 가까운 문장은 {source}에서 찾음.")
    print(f"거리: {combined_distance:.4f}")  # 거리 값 그대로 출력
    print(f"내용: {result_text}")
    
    return result_text, source

# 함수 호출 예시
search_combined_index(query='2023년 한국은행의 정책목표는?')